In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:

 

1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






Run following in local Jupyter

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


Run following 2 cells for colab to import libraries

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


Run following if the library code is modified locally

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [19]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [8]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
#            'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
#            'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
#            'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
#            'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
#            'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
#            'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
#            'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM', 'RBLX',
#            'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'STNE','SWAV',
#            'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
#            'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
model = "gstock-4a-test"
tickers = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [ ]:
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)


In [23]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc,Gain_f
2,ALSMY,1.63,0.88,18.44,25.98,2.36,3.010000,0.27,0.69,15.31,23.56,2.990000,0.27,0.76,20.56,29.23,2.420000,0.03,8.355556,0.27
13,CPG,0.75,0.87,49.04,46.08,7.78,5.670000,-0.27,0.85,75.00,51.54,7.370000,-0.05,0.71,45.21,43.48,6.120000,-0.21,8.080952,-0.21
53,SFTBY,13.39,0.91,181.50,190.92,19.03,26.780001,0.41,0.71,128.89,162.18,22.420000,0.18,0.70,153.99,197.95,18.780001,-0.01,8.600000,0.18
43,NU,3.45,0.84,12.65,17.11,4.66,5.970000,0.28,0.67,10.34,12.76,5.480000,0.18,0.67,15.99,15.91,4.460000,-0.04,7.511111,0.18
39,LVMUY,61.58,0.87,721.41,528.83,196.59,167.240005,-0.15,0.78,721.97,434.18,193.550003,-0.02,0.70,642.68,385.77,169.160004,-0.14,7.771429,-0.14
38,LYB,30.34,0.89,387.02,532.12,96.85,84.480003,-0.13,0.82,657.76,579.68,95.199997,-0.02,0.72,547.77,520.67,80.779999,-0.17,8.053846,-0.13
48,POAHF,34.74,0.88,429.42,560.01,58.69,66.260002,0.13,0.78,497.16,420.06,66.690002,0.14,0.78,455.73,609.35,55.430000,-0.06,7.876923,0.13
5,ADDDF,98.08,0.90,1271.27,2008.57,176.00,209.389999,0.19,0.80,1727.46,1907.72,197.940002,0.12,0.69,1468.32,1843.66,169.529999,-0.04,8.166667,0.12
6,ADYEY,6.87,0.90,137.34,111.89,15.88,13.120000,-0.17,0.70,110.51,88.58,16.680000,0.05,0.63,113.39,106.23,14.020000,-0.12,8.166667,-0.12
26,FQVLF,4.13,0.86,201.44,118.50,25.79,22.469999,-0.13,0.76,224.66,197.53,24.910000,-0.03,0.62,151.02,133.57,21.790001,-0.16,7.523077,-0.13


In [ ]:
df1  = pipeline.runModelCombinedVolaR(tickers, model, mod, False, loss=lossfn)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False).head(50)


,,Name,Error,Accu,Buy,Sell,Total,Last,Pred,Gain,Alloc
Ticker,Target,,,,,,,,,,
PROSF,adjclose,RROnly,43.62,0.97,537.58,466.35,1003.94,71.27,81.980003,0.15,9.500000
ADYEY,adjclose,RROnly,6.69,0.96,158.37,129.34,287.71,15.88,14.230000,-0.10,9.200000
COTY,adjclose,RROnly,2.84,0.94,83.03,89.97,173.01,12.26,10.630000,-0.13,8.938462
NOK,adjclose,RROnly,2.47,0.94,27.42,36.66,64.09,4.90,5.450000,0.11,8.854545
STLA,adjclose,RROnly,5.24,0.94,111.36,84.21,195.57,18.86,16.700001,-0.11,8.854545
TKAMY,adjclose,RROnly,3.99,0.94,82.39,90.40,172.78,7.83,8.600000,0.10,8.800000
NU,adjclose,RROnly,3.47,0.93,14.38,26.09,40.47,4.66,5.220000,0.12,8.716667
CPG,adjclose,RROnly,0.72,0.90,62.35,36.14,98.49,7.78,5.590000,-0.28,8.642857
COVTY,adjclose,RROnly,11.97,0.92,125.21,160.04,285.25,20.17,22.510000,0.12,8.533333


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV', 
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE', 
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE', 
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [10]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 

sending incremental file list
gstock-4a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
gstock-4a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
gstock-4a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,640,238 bytes  received 73 bytes  6,426,874.00 bytes/sec
total size is 9,637,296  speedup is 1.00
